In [55]:
from __future__ import annotations
from IPython.display import display
import numpy as np
from typing import List, Iterable, cast
from dataclasses import dataclass

In [98]:
@dataclass
class Car:
    car_id: int
    school_id: int
    car_name: str
    car_scruitineered: bool = False
    present_round_robin: bool = False
    present_knockout: bool = False
    points: int = 0 # Lower is better.
    
    def __repr__(self) -> str:
        return f"<{self.car_id:>3d}, {self.points:>2d}>"

In [99]:
class Race:
    def __init__(self, left_seed: int, right_seed: int, winner_next_race: Race | None):
        self.left_seed = left_seed
        self.right_seed = right_seed
        self.winner_next_race: Race | None = winner_next_race
        self.left_car: Car | None = None
        self.right_car: Car | None = None

    def theoretical_winner(self) -> int:
        return min(self.left_seed, self.right_seed)

    def theoretical_loser(self) -> int:
        return max(self.left_seed, self.right_seed)

    def __repr__(self) -> str:
        def car_none_str(car_none: Car | None):
            """Calls repr if this is a car, puts a placeholder in otherwise."""
            if car_none is None:
                return "<___, __>"
            else:
                return repr(car_none)

        return f"({self.left_seed:>2d} {car_none_str(self.left_car)}, {self.right_seed:>2d} {car_none_str(self.right_car)})"

In [100]:
competitors = 12
rounds = int(np.ceil(np.log2(competitors)))
grand_final = Race(1, 2, None)

In [101]:
def add_round(next_round: List[Race]) -> List[Race]:
    races = []
    competitors_in_round = 4 * len(next_round)

    def seed_pair(seed: int) -> int:
        """Returns the pair of a seed.
        (The worst opponent for the current rank).
        The sum of the pair should add to the number of competitors in the round + 1."""
        return competitors_in_round + 1 - seed

    for next_round_race in next_round:
        high_seed = next_round_race.theoretical_winner()
        races.append(
            Race(
                left_seed=high_seed,
                right_seed=seed_pair(high_seed),
                winner_next_race=next_round_race,
            )
        )
        low_seed = next_round_race.theoretical_loser()
        races.append(
            Race(
                left_seed=low_seed,
                right_seed=seed_pair(low_seed),
                winner_next_race=next_round_race,
            )
        )

    return races

In [102]:
event: List[List[Race]] = [[grand_final]]
for event_round in range(rounds - 2, -1, -1):
    event.append(add_round(event[-1]))

# Flip the order so that the first round is at the start.
event = list(reversed(event))

In [103]:
def print_event(event:Iterable[List[Race]]) -> None:
    for round_num, r in enumerate(event):
        print(f"{round_num:5}: {r}")

print_event(event)

    0: [( 1 <___, __>, 16 <___, __>), ( 8 <___, __>,  9 <___, __>), ( 4 <___, __>, 13 <___, __>), ( 5 <___, __>, 12 <___, __>), ( 2 <___, __>, 15 <___, __>), ( 7 <___, __>, 10 <___, __>), ( 3 <___, __>, 14 <___, __>), ( 6 <___, __>, 11 <___, __>)]
    1: [( 1 <___, __>,  8 <___, __>), ( 4 <___, __>,  5 <___, __>), ( 2 <___, __>,  7 <___, __>), ( 3 <___, __>,  6 <___, __>)]
    2: [( 1 <___, __>,  4 <___, __>), ( 2 <___, __>,  3 <___, __>)]
    3: [( 1 <___, __>,  2 <___, __>)]


In [105]:
def assign_cars(cars:List[Car], first_round:List[Race]) -> None:
    """Assigns cars to the first round of the draw."""
    sorted_cars: List[Car|None] = sorted(cars, key=lambda c: cast(Car,c).points, reverse=False) # Set reverse to True to reward higher rather than lower points.
    byes = 2*len(first_round) - len(sorted_cars)
    sorted_cars.extend([None] * byes)
    assert len(sorted_cars) == 2*len(first_round), "We should have introduced enough byes to obtain the required number of participents, but something went wrong."

    for race in first_round:
        race.left_car = sorted_cars[race.left_seed-1]
        race.right_car = sorted_cars[race.right_seed-1]

cars = [
    Car(400+i, 0, f"Test car {i}", points = i) for i in range(1, 14) 
]

assign_cars(cars, event[0])
print_event(event)

    0: [( 1 <401,  1>, 16 <___, __>), ( 8 <408,  8>,  9 <409,  9>), ( 4 <404,  4>, 13 <413, 13>), ( 5 <405,  5>, 12 <412, 12>), ( 2 <402,  2>, 15 <___, __>), ( 7 <407,  7>, 10 <410, 10>), ( 3 <403,  3>, 14 <___, __>), ( 6 <406,  6>, 11 <411, 11>)]
    1: [( 1 <___, __>,  8 <___, __>), ( 4 <___, __>,  5 <___, __>), ( 2 <___, __>,  7 <___, __>), ( 3 <___, __>,  6 <___, __>)]
    2: [( 1 <___, __>,  4 <___, __>), ( 2 <___, __>,  3 <___, __>)]
    3: [( 1 <___, __>,  2 <___, __>)]
